### Census Trade Data

September 30, 2020

Brian Dew

In [1]:
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

from io import BytesIO, TextIOWrapper
from zipfile import ZipFile

In [2]:
# Retrieve data from Census API
d = {}
for flow in ['imports', 'exports']:
    base = f'https://api.census.gov/data/timeseries/intltrade/{flow}/hs'
    params = ['CTY_CODE', 'CTY_NAME', 'ALL_VAL_MO']
    if flow == 'imports':
        params[2] = 'GEN_VAL_MO'
    param = ','.join(params)
    url = f'{base}?get={param}&key={census_key}&time=from+1992-01'
    d[flow] = requests.get(url).json()
    
i = d['imports']
df = pd.DataFrame(i[1:], columns=i[0])
df = df[(df['CTY_CODE'].str[0] !='0') & 
        (df['CTY_CODE'].str[1] != 'X')]
df['Name'] = df['CTY_NAME'].str.title()
df['Date'] = pd.to_datetime(df['time'], format='%Y-%m')
imp = (df.rename({'GEN_VAL_MO': 'Imports'}, axis=1)
         .set_index(['Name', 'Date']).Imports.astype(float))

e = d['exports']
df = pd.DataFrame(e[1:], columns=e[0])
df = df[(df['CTY_CODE'].str[0] !='0') & 
        (df['CTY_CODE'].str[1] != 'X')]
df['Name'] = df['CTY_NAME'].str.title()
df['Date'] = pd.to_datetime(df['time'], format='%Y-%m')
exp = (df.rename({'ALL_VAL_MO': 'Exports'}, axis=1)
         .set_index(['Name', 'Date']).Exports.astype(float))

data = ((pd.DataFrame([imp, exp]).T / 1000000)
           .rename({'Korea, South': 'South Korea',
                    'Total For All Countries': 'Total, All Countries'}))

data.to_csv(data_dir / 'trade_master.csv')

In [3]:
data = pd.read_csv(data_dir / 'trade_master.csv', 
                   parse_dates=True, index_col=['Name', 'Date'])

data['Total'] = data['Imports'] + data['Exports']

dt = data.xs('Canada').index[-1]
ltdate = dtxt(dt)['mon1']
index = data.xs(dt, level=1).Total.sort_values(ascending=False).iloc[:26].index

final = data.loc[(index, dt),:].astype(int).droplevel('Date')
dt2 = dt - pd.DateOffset(months = 12)
data2 = data.loc[(index, dt2),:].astype(int).droplevel('Date')
data2.columns = [f'{k} ' for k in final.keys()]
final = final.join(data2).sort_values('Total', ascending=False)
final.index.name = ''

(final.applymap('{:,.0f}'.format)
      .to_csv(data_dir / 'trade_partner.tex', sep='&', 
              lineterminator='\\\ ', quotechar=' '))

write_txt(text_dir / 'trade_date.txt', dtxt(dt)['mon1'])
write_txt(text_dir / 'trade_date2.txt', dtxt(dt2)['mon1'])

shval = (final['Total'].iloc[1:].sum() / 
         final['Total'].iloc[0]) * 100
tc = final.drop('Total, All Countries').index[:3]
tc_txt = f'{tc[0]}, {tc[1]}, and {tc[2]}'
t3sh = (final.loc[tc, 'Total'].sum() / 
        final.loc['Total, All Countries', 'Total'])
t3sh_txt = f'{t3sh*100:.1f} percent'

text = (f'In {ltdate}, trade with the top 25 trading partners (see '+
        f'table) comprises {shval:.1f} percent of total US trade in '+
        'goods. The top three US trading partners are '+
        f'{tc_txt}. These three countries account for {t3sh_txt} of '+
        f'US goods trade in {ltdate}. ')
write_txt(text_dir / 'trade_partner_sh.txt', text)
print(text)

In March 2023, trade with the top 25 trading partners (see table) comprises 85.3 percent of total US trade in goods. The top three US trading partners are Mexico, Canada, and China. These three countries account for 41.7 percent of US goods trade in March 2023. 


### Trade Levels, Latest in Detail

In [4]:
url = 'https://www.census.gov/econ_getzippedfile/?programCode=FTD'
r = requests.get(url)
zip_file = ZipFile(BytesIO(r.content))
with zip_file.open('FTD-mf.csv', 'r') as csvfile:
    tables = csvfile.read().split(b"\n\n\n")
    tables = [tbl.decode().replace('\r', '').split('\n') for tbl in tables]

In [5]:
d = {tbl[0]: pd.DataFrame([t.split(',') for t in tbl[1:]]).T.set_index(0).T 
     for tbl in tables}
cats = (d['CATEGORIES'].loc[:, ['cat_idx', 'cat_code']]
        .set_index('cat_idx').to_dict()['cat_code'])
tp = (d['TIME PERIODS'].loc[:, ['per_idx', 'per_name']]
        .set_index('per_idx').to_dict()['per_name'])
dtype = (d['DATA TYPES'].loc[:, ['dt_idx', 'dt_code']]
        .set_index('dt_idx').to_dict()['dt_code'])
data = pd.DataFrame()
data['CAT'] = d['DATA']['cat_idx'].dropna().map(cats)
data['DATE'] = d['DATA']['per_idx'].dropna().map(tp)
data['TYPE'] = d['DATA']['dt_idx'].dropna().map(dtype)
data['DATA'] = d['DATA']['val'].dropna().astype('int') / 1000 
data['SA'] = d['DATA']['is_adj']
data.index = pd.to_datetime(data.DATE)
df = pd.DataFrame()
for i in ['IMP', 'EXP', 'BAL']:
    df[i] = data.query('CAT == "BOPGS" & SA == "1" & TYPE == @i')['DATA']
df.loc['2015':].to_csv(data_dir / 'tradelt.csv', index_label='date')

In [6]:
# result = pd.DataFrame()
# url0 = 'https://www.census.gov/econ/currentdata/export/csv'
# url1 = ('?programCode=FTD&timeSlotType=12&startYear=2015&'+
#         'endYear=2022&categoryCode=BOPGS&')
# for flow in ['EXP', 'IMP']:
#     url2 = (f'dataTypeCode={flow}&geoLevelCode=US&adjusted=yes&'+
#             'errorData=no&internal=false')
#     url = url0 + url1 + url2
#     s = pd.read_csv(url, skiprows=6).set_index('Period').dropna()
#     s.index = pd.to_datetime(s.index)
#     result[flow] = s['Value']

# balance = lambda x: x.EXP - x.IMP    
# df = result.loc['2015':].div(1000).assign(BAL = balance)
# df.to_csv(data_dir / 'tradelt.csv', index_label='date')

In [7]:
# Chart and text for latest monthly trade data
df = pd.read_csv(data_dir / 'tradelt.csv', index_col='date', 
                 parse_dates=True)

# End nodes for time series graph
res = df[['IMP', 'EXP']]
adj = node_adj(res)
#smax = res2.iloc[-1].idxmax()
#adj[smax] = adj[smax] + 0.35
cols = {'IMP': 'blue!80!violet', 'EXP': 'green!80!blue'}
date = {series: 'm' if series in ['IMP'] else None 
        for series in cols.keys()}
nodes  ='\n'.join([end_node(res[series], color, date=date[series], 
                            full_year=True, offset=adj[series]) 
                   for series, color in cols.items()])
write_txt(text_dir / 'tradelt_nodes.txt', nodes) 
node = end_node(df.BAL, 'red', date='m', offset=0.35, 
                full_year=True)
write_txt(text_dir / 'ballt.txt', node)

ltdt = dtxt(df.index[-1])['mon1']
ltmo = dtxt(df.index[-1])['mon3']
prdt = (dtxt(df.index[-2])['mon3'] 
        if df.index[-1].year == df.index[-2].year 
        else dtxt(df.index[-2])['mon1'])
dfs = [('lt', df.iloc[-1]), ('pr', df.iloc[-2]), 
       ('lt3', df.iloc[-3:].mean()), 
       ('pr3', df.iloc[-15:-12].mean()), 
       ('pr19', df.loc['2019'].mean())]
val = {name: srs.apply(lambda x: f'\${abs(x):.1f} billion') 
       for name, srs in dfs}
cols = [('IMP', 'blue!80!violet'), ('EXP', 'green!80!blue'),
        ('BAL', 'red')]
cline = {name: c_line(col) for name, col in cols}

# Context values
pop = (pd.read_csv(data_dir/'nipa20600.csv', index_col='date', 
                  parse_dates=True)['B230RC'])

emp = (pd.read_csv(data_dir / 'jobs_report_main2.csv', 
                   index_col='date', parse_dates=True)
        .loc['1989':, 'EMPsa'])

tpop = (df.multiply(1_000_000).divide(pop, axis=0).dropna()
          .applymap('\${:,.0f} per capita'.format))
temp = (df.multiply(1_000_000).divide(emp, axis=0).dropna()
          .applymap('\${:,.0f} per worker'.format))
imp_pc = tpop['IMP'].iloc[-1]
exp_pc = tpop['EXP'].iloc[-1]
exp_pw = temp['EXP'].iloc[-1]

text = (f'US goods and services imports total {val["lt"].IMP} in {ltdt}, '+
        f'following {val["pr"].IMP} in {prdt} {cline["IMP"]}. '+
        f'Imports average {val["lt3"].IMP} over the latest three months of data, '+
        f'and {val["pr3"].IMP} during the same months, one year prior. '+
        f'In 2019, monthly US imports average {val["pr19"].IMP}. For additional '+
        f'context, imports are equivalent to {imp_pc}, in '+
        'the latest month.\n\n'+
        f'The US exported {val["lt"].EXP} of goods and services in {ltdt}, '+
        f'following {val["pr"].EXP} in {prdt} {cline["EXP"]}. The three-month '+
        f'average was {val["lt3"].EXP} in {ltmo}, and {val["pr3"].EXP} '+
        f'one year prior. Exports were {val["pr19"].EXP} per month, '+
        'on average, in 2019. In the latest month, exports are equivalent '+
        f'to {exp_pc} or {exp_pw}. \n\n'+
        'Spending on imports exceeds payments '+
        f'received for exports, resulting in a trade deficit. In {ltmo}, '+
        f'the trade deficit was {val["lt"].BAL}, following {val["pr"].BAL} '+
        f'in {prdt} {cline["BAL"]}. Over the past three months, the average '+
        f'trade deficit is {val["lt3"].BAL}, compared to '+
        f'{val["pr3"].BAL} one year prior. '+
        f'In 2019, the average monthly trade deficit is {val["pr19"].BAL}.')
write_txt(text_dir / 'tradeltlevels.txt', text)
print(text)

US goods and services imports total \$320.4 billion in March 2023, following \$321.5 billion in February (see {\color{blue!80!violet}\textbf{---}}). Imports average \$322.8 billion over the latest three months of data, and \$328.2 billion during the same months, one year prior. In 2019, monthly US imports average \$258.8 billion. For additional context, imports are equivalent to \$957 per capita, in the latest month.

The US exported \$256.1 billion of goods and services in March 2023, following \$250.8 billion in February (see {\color{green!80!blue}\textbf{---}}). The three-month average was \$255.0 billion in March, and \$234.5 billion one year prior. Exports were \$212.2 billion per month, on average, in 2019. In the latest month, exports are equivalent to \$765 per capita or \$1,592 per worker. 

Spending on imports exceeds payments received for exports, resulting in a trade deficit. In March, the trade deficit was \$64.2 billion, following \$70.6 billion in February (see {\color{r

### Monthly trade data summary table

In [8]:
url = ('https://www.census.gov/foreign-trade/Press-Release/current_press_release/exh1.xlsx')
df = pd.read_excel(url, skipfooter=5, header=[4,5], index_col=0)
df = df.loc[~df.index.str.contains('-')]
y1 = df.index.name
y2 = df.index[12]
y3 = df.index[25]
i1 = df.index[:12] + ' 01, ' + y1
i2 = df.index[13:25]  + ' 01, ' + y2
i3 = df.iloc[26:].dropna().index.str.replace(' (R)', '', regex=False)  + ' 01, ' + y3

df2 = df.dropna()
df2.index = pd.to_datetime([*i1, *i2, *i3])
df2.columns = [i[1] + ' ' + i[0] for i in df2.columns]
df2.columns = df2.columns.str.replace(' (1)', '', regex=False)
df2.to_csv(data_dir / 'trade_mo_summary.csv', index_label='date')

In [9]:
lt = df2.iloc[-3:].iloc[::-1].T
pryr = df2.iloc[-13].to_frame()
modt = pd.concat([lt, pryr], axis=1)
cm = (df2.resample('QS').count() > 2)['Total Balance']
ltqtr = df2.resample('QS').mean().loc[cm[cm].index[-1]].to_frame()
prqtr = df2.resample('QS').mean().loc[cm[cm].index[-2]].to_frame()
qtrdt = pd.concat([ltqtr, prqtr], axis=1)
modt.columns = [dtxt(i)['mon2'] for i in modt.columns]
qtrdt.columns = [dtxt(i)['qtr1'] for i in qtrdt.columns]
table = pd.concat([modt, qtrdt], axis=1)
table = table.applymap('{:,.0f}'.format)
(table.to_csv(data_dir / 'trade_mo_summary.tex', sep='&', 
            lineterminator='\\\ ', quotechar=' '))

### World Bank Tariff Data (WDI)

In [10]:
from io import BytesIO
from zipfile import ZipFile

url = 'http://api.worldbank.org/v2/en/indicator/TM.TAX.MRCH.WM.AR.ZS?downloadformat=csv'

r = requests.get(url)
zip_file = ZipFile(BytesIO(r.content))
files = zip_file.namelist()
data_file = [file for file in files if file[0] != 'M']
with zip_file.open(data_file[0], 'r') as csvfile:
    df = pd.read_csv(csvfile, skiprows=4)
    
data = df[df['Country Code'] == 'USA'].T.loc['1989':].dropna().iloc[:,0]
data.index = pd.to_datetime([f'{year}-07-01' for year in data.index])
data.name = 'tariff'
data.index.name = 'date'
data[pd.to_datetime('1994-07-01')] = np.nan
data = data.sort_index()
data.to_csv(data_dir / 'tariff.csv')
write_txt(text_dir / 'tariff_node.txt', end_node(data, 'violet', percent=True))

t89 = data.loc['1989-07-01']
tlt = data.iloc[-1]
ltyear = dtxt(data.index[-1])['year']

text = ('World Bank staff \href{https://data.worldbank.org/indicator/TM.TAX.MRCH.WM.AR.ZS?locations=US&name_desc=true}{calculate} '+
        'the weighted average applied tariff rate on US imports. The measure shows the trend '+
        'in applied tariffs, factoring in the actual composition of imports by trading partner. '+
        f'In 1989, the average tariff on US imports was {t89:.1f} percent. During the 1990s and 2000s, '+
        'the US entered into various trade agreements, while China joined the WTO, obtaining '+
        '\href{https://www.wto.org/english/thewto_e/whatis_e/tif_e/fact2_e.htm}{most-favored nation} '+
        f'status with the US. In the latest data, covering {ltyear}, the average tariff was {tlt:.1f} '+
        'percent.')
write_txt(text_dir / 'tariff.txt', text)
print(text)

World Bank staff \href{https://data.worldbank.org/indicator/TM.TAX.MRCH.WM.AR.ZS?locations=US&name_desc=true}{calculate} the weighted average applied tariff rate on US imports. The measure shows the trend in applied tariffs, factoring in the actual composition of imports by trading partner. In 1989, the average tariff on US imports was 3.9 percent. During the 1990s and 2000s, the US entered into various trade agreements, while China joined the WTO, obtaining \href{https://www.wto.org/english/thewto_e/whatis_e/tif_e/fact2_e.htm}{most-favored nation} status with the US. In the latest data, covering 2020, the average tariff was 1.5 percent.
